In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from time import time

In [ ]:
#encoding:utf-8
###########import packages##########
import shap
import tensorflow as tf
import keras
from keras import optimizers
from keras import regularizers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.constraints import max_norm
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.models import Model
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xgboost 
import lightgbm
import catboost
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import ensemble
from sklearn.tree import ExtraTreeRegressor
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score
%matplotlib
###########wrapping root mean square error for later calls##########
def compute_mae_mse_rmse(target,prediction):
    error = []
    for i in range(len(target)):
        error.append(target[i] - prediction[i])
    squaredError = []
    absError = []
    for val in error:
        squaredError.append(val * val)  # target-prediction之差平方
        absError.append(abs(val))  # 误差绝对值
    mae=sum(absError)/len(absError)  # 平均绝对误差MAE
    mse=sum(squaredError)/len(squaredError)  # 均方误差MSE
    RMSE=np.sqrt(sum(squaredError)/len(squaredError))
    R2=r2_score(target,prediction)
    return mae,mse,RMSE,R2
###########loading data##########
fdata=pd.read_csv('datatest.csv',encoding="gbk")
raw_data=fdata.loc[:,['Membrane Thickness (μm)',#0
                      'GDE(1 for GDE 0 for CCM)',#1
                      'Dielectric Constant (C  N-1 M-2)',#2
                      'Flash Point(℃)',#3
                      'Stirring Method (0 for Mechanical Stirring 1 for Ultrasound)',#4
                      'Stirring Temp (℃)',#5
                      'Flow Rate (mL min-1)',#6
                      'Water Content (wt%)',#7
                      'Nafion EW (mol g-1)',#8
                      'Anodic Platinum Loading Amount (mgPt cm-2)',#9
                      'Cathodic Platinum Loading Amount (mgPt cm-2)',#10
                      'Drying Temperature (℃)',#11
                      'I/C',#12
                      'Cell Temperature (℃)',#13
                      'Anode Flow Rate (sccm)',#14
                      'Cathode Flow Rate (sccm)',#15
                      'Active Area (cm2)',#16
                      'Solid Content (wt%)',#17
                      'Backpressure (Mpa)',#18
                      'PTUTIL_CL',#19
                      'PD06_CL'
                        ]]



###########defining a wrapper function for later call from each machine learning algorithms##########
raw_input=raw_data.iloc[:,0:19]
raw_output_PC=raw_data.iloc[:,19]
raw_output_PD=raw_data.iloc[:,20]
# raw_input_global=raw_data.iloc[:,0:32]
# raw_output_global=raw_data.iloc[:,32]
# X=raw_input.values.astype(np.float32)
# y=raw_output.values.astype(np.float32)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
def plot_3d_tsne(dataset,classification_result):
    cl=classification_result.values
    colors = ['red', 'blue']
    # 根据主成分分析结果绘制散点图
    fig=plt.figure()
    ax=Axes3D(fig)
    for i in range(len(colors)):
        x = dataset[:, 0][cl == i]
        y = dataset[:, 1][cl == i]
        z = dataset[:, 2][cl == i]
        ax.scatter(x,y,z,c=colors[i])
#         ax.set_zlabel('Third Principal Component')
        ax.set_zlabel('Z')
    # 设置图例，0-9用不同颜色表示
    plt.legend(np.array([0,1]), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    # 设置坐标标签
#     plt.xlabel('First Principal Component')
#     plt.ylabel('Second Principal Component')
    plt.xlabel('X')
    plt.ylabel('Y')
    # 设置标题
    plt.title("t-SNE Scatter Plot 3D")
    plt.show()
def plot_2d_tsne(dataset,classification_result):
    cl=classification_result.values
    colors = ['red', 'blue']
    # 根据主成分分析结果绘制散点图
    fig=plt.figure()
    for i in range(len(colors)):
        x = dataset[:, 0][cl == i]
        y = dataset[:, 1][cl == i]
        plt.scatter(x,y,c=colors[i])
    # 设置图例，0-9用不同颜色表示
    plt.legend(np.array([0,1]), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    # 设置坐标标签
#     plt.xlabel('First Principal Component')
#     plt.ylabel('Second Principal Component')
    plt.xlabel('X')
    plt.ylabel('Y')
    # 设置标题
    plt.title("t-SNE Scatter Plot 2D")
    # 显示图形
    plt.show()

In [ ]:
print('Computing t-SNE embedding')
tsne_2 = TSNE(n_components=2, random_state=0,init='random',n_iter=10000)
result_2 = tsne_2.fit_transform(raw_input)
print(result_2.shape)
plot_2d_tsne(result_2,raw_output_PC)

In [ ]:
print('Computing t-SNE embedding')
tsne_3 = TSNE(n_components=3, random_state=0,init='random',n_iter=10000)
result_3 = tsne_3.fit_transform(raw_input)

plot_3d_tsne(result_3,raw_output_PC)

In [ ]:
print('Computing t-SNE embedding')
tsne_2 = TSNE(n_components=2, random_state=0,init='random',n_iter=10000)
result_2 = tsne_2.fit_transform(raw_input)
print(result_2.shape)
plot_2d_tsne(result_2,raw_output_PD)

In [ ]:
print('Computing t-SNE embedding')
tsne_3 = TSNE(n_components=3, random_state=0,init='random',n_iter=10000)
result_3 = tsne_3.fit_transform(raw_input)

plot_3d_tsne(result_3,raw_output_PD)